In [1]:
def echoDensity(self, IR, N, fs, preDelay):
        mixingThresh = 1.0
        s = np.zeros(len(IR))
        echo_dens = np.zeros(len(IR))

        wTau = signal.hann(N).T
        wTau = wTau / sum(wTau)
        halfWin = round(N / 2)

        sparseInd = np.arange(0, len(IR), 500)

        for n in sparseInd:
            if n <= halfWin + 1:
                hTau = IR[0:n + halfWin].T
                # end = int(wTau[-1])
                wT = wTau[(-1 - halfWin - n + 1):]

            elif (n > halfWin+1) and (n <= len(IR)-halfWin+1):
                hTau = IR[(n - halfWin):(n + halfWin)].T
                wT = wTau

            elif n > len(IR)-halfWin+1:
                hTau = IR[n - halfWin:].T
                wT = wTau[0:len(hTau)]

            s[n] = np.sqrt(np.dot(wT, (hTau * hTau)))

            tipCt = np.zeros(len(hTau))
            for i in range(len(hTau)):
                if abs(hTau[i]) > s[n]:
                    tipCt[i] = 1
                else:
                    tipCt[i] = 0

            echo_dens[n] = np.dot(wT, tipCt.T)

        echo_dens = echo_dens / erfc(1 / sqrt(2))
        fill = np.arange(0, len(IR))
        a = (echo_dens[sparseInd])
        echo_dens_new = interpolate.interp1d(sparseInd, echo_dens[sparseInd],axis=0, fill_value="extrapolate")
        # d = np.argwhere(echo_dens > mixingThresh)[0]
        # t_abel = (d -preDelay)/fs * 1000

        return echo_dens_new(fill)